In [1]:
# Import the dependencies
import pandas as pd
import gmaps
import requests

# Import the API key
from config import g_key


In [2]:
# Store the CSV you saved created in part one into a DataFrame
city_data_df = pd.read_csv("weather_data/cities.csv")
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Luau,-10.7073,22.2247,67.82,92,100,3.22,AO,2022-01-09 05:35:10
1,1,Nanortalik,60.1432,-45.2371,14.68,80,100,17.49,GL,2022-01-09 05:32:09
2,2,Caarapo,-22.6342,-54.8222,66.74,86,31,6.02,BR,2022-01-09 05:35:10
3,3,Ushuaia,-54.8000,-68.3000,40.32,94,47,3.62,AR,2022-01-09 05:26:59
4,4,Ilebo,-4.3167,20.5833,73.54,96,100,0.36,CD,2022-01-09 05:35:11


In [3]:
# Get the data types
city_data_df.dtypes

City_ID         int64
City           object
Lat           float64
Lng           float64
Max Temp      float64
Humidity        int64
Cloudiness      int64
Wind Speed    float64
Country        object
Date           object
dtype: object

In [4]:
# Configure gmaps to use the Google API key
gmaps.configure(api_key=g_key)

In [5]:
# 1. Assign the locations to an array of latitude and logitude pairs
locations = city_data_df[["Lat", "Lng"]]
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
temps = []
for temp in max_temp:
    temps.append(max(temp, 0))
# 3. Assign the figure variable to the gmaps.figure() attribute
fig = gmaps.figure()
# 4. Assign the heatmap_layer variable to the heatmap_layer attribute
heat_layer = gmaps.heatmap_layer(locations, weights=
                                 [max(temp, 0) for temp in max_temp], 
                                 dissipating=False, max_intensity=300, 
                                 point_radius=4) 
# 5. Add the heatmap layer
fig.add_layer(heat_layer)
# 6. Call the figure to plot the data
fig

Figure(layout=FigureLayout(height='420px'))

In [6]:
# Heatmap of percent humidity
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [7]:
# Heatmap of percent cloudiness
locations = city_data_df[["Lat", "Lng"]]
clouds = city_data_df["Cloudiness"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=clouds, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [8]:
# Heatmap of percent cloudiness
locations = city_data_df[["Lat", "Lng"]]
wind = city_data_df["Wind Speed"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=wind, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [9]:
# Ask the customer to add a minimum and maximum temp value
min_temp = float(input("What is the minimum temperature you would like for your trip?"))
max_temp = float(input("What is the maximum temperature you would like for your trip?"))

What is the minimum temperature you would like for your trip?60
What is the maximum temperature you would like for your trip?85


In [10]:
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Luau,-10.7073,22.2247,67.82,92,100,3.22,AO,2022-01-09 05:35:10
2,2,Caarapo,-22.6342,-54.8222,66.74,86,31,6.02,BR,2022-01-09 05:35:10
4,4,Ilebo,-4.3167,20.5833,73.54,96,100,0.36,CD,2022-01-09 05:35:11
7,7,Saint-Philippe,-21.3585,55.7679,83.75,77,12,6.06,RE,2022-01-09 05:30:36
9,9,Port Alfred,-33.5906,26.8910,74.46,86,40,22.44,ZA,2022-01-09 05:27:56
10,10,Busselton,-33.6500,115.3333,75.09,35,1,15.37,AU,2022-01-09 05:30:28
11,11,Hermanus,-34.4187,19.2345,74.70,72,0,5.44,ZA,2022-01-09 05:32:08
14,14,Codrington,-38.2667,141.9667,65.35,62,5,14.56,AU,2022-01-09 05:35:13
16,16,French Harbor,16.3500,-86.4333,79.00,87,75,3.00,HN,2022-01-09 05:35:13
17,17,Victoria,22.2855,114.1577,70.25,78,77,1.01,HK,2022-01-09 05:31:14


In [11]:
preferred_cities_df.count()


City_ID       282
City          282
Lat           282
Lng           282
Max Temp      282
Humidity      282
Cloudiness    282
Wind Speed    282
Country       275
Date          282
dtype: int64

In [12]:
# Create DataFrame called hotel_df to store hotel names along with
#city, country, max temp, and coordinates
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)


,City,Country,Max Temp,Lat,Lng,Hotel Name
0,Luau,AO,67.82,-10.7073,22.2247,
2,Caarapo,BR,66.74,-22.6342,-54.8222,
4,Ilebo,CD,73.54,-4.3167,20.5833,
7,Saint-Philippe,RE,83.75,-21.3585,55.7679,
9,Port Alfred,ZA,74.46,-33.5906,26.8910,
10,Busselton,AU,75.09,-33.6500,115.3333,
11,Hermanus,ZA,74.70,-34.4187,19.2345,
14,Codrington,AU,65.35,-38.2667,141.9667,
16,French Harbor,HN,79.00,16.3500,-86.4333,
17,Victoria,HK,70.25,22.2855,114.1577,


In [13]:
# Set the parameters to search for a hotel in Paris.
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key,
    "location": "48.8566, 2.3522"}
# Use base URL to search for hotels in Paris.
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
# Make request and get the JSON data from the search.
hotels = requests.get(base_url, params=params).json()

hotels

{'html_attributions': [],
 'next_page_token': 'Aap_uEDlQ2UTrB3S7PhOvvNG-D7iY_JfG9bRlH8nnsSCj-4_3Yy9a-GqGis931oS33Qa_FCBOJdd01MeJrCbrO4qL4QAGCcqIQfA_BdOuMBArDcbsATpLnNXZcjcGNrIalamkdRytQZCF-bl5U8HhYKQ9RYpaeX4jYh7l051Gr_5Va28yCM5jaeg-lEjvCd5WfnDxhbsWG5katNhpioYSEv9UwMlZqf0o4HQ2V-CP4QsCvFxd9VZM7JsyIhu2eISKzChJMyXwEC4EgkOBobNtMgQE6m45a3YKZ8V8RcNddC2NZ-dM_3B000ichm5FRh7Y25rQJXPxYaLsVi2saiM6G73lHjyDLR31qhgUEOp2uCRyTQ1nXJT_nj9HPVeLyFGutnCXCJTaKr3YswYD8FrKaxRwCUJ3i-Lwy0iMl66sESQzyBJbVrL6ltXMkyT',
 'results': [{'business_status': 'OPERATIONAL',
   'geometry': {'location': {'lat': 48.8581126, 'lng': 2.3529277},
    'viewport': {'northeast': {'lat': 48.8594360302915,
      'lng': 2.354355480291502},
     'southwest': {'lat': 48.8567380697085, 'lng': 2.351657519708498}}},
   'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png',
   'icon_background_color': '#909CE1',
   'icon_mask_base_uri': 'https://maps.gstatic.com/mapfiles/place_api/icons/v2/hotel_pinlet',
   'nam

In [14]:
len(hotels["results"])

20

In [15]:
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
hotel_df.head(10)     



Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


,City,Country,Max Temp,Lat,Lng,Hotel Name
0,Luau,AO,67.82,-10.7073,22.2247,Casa- Bernardo Cândido
2,Caarapo,BR,66.74,-22.6342,-54.8222,Hotel Pousada Quinta do Sul
4,Ilebo,CD,73.54,-4.3167,20.5833,Hôtel BUMAK
7,Saint-Philippe,RE,83.75,-21.3585,55.7679,"Chambres d'hôte ""La Trinité"""
9,Port Alfred,ZA,74.46,-33.5906,26.8910,The Halyards Hotel
10,Busselton,AU,75.09,-33.6500,115.3333,Observatory Guest House
11,Hermanus,ZA,74.70,-34.4187,19.2345,Misty Waves Boutique Hotel
14,Codrington,AU,65.35,-38.2667,141.9667,Codrington Gardens Bed And Breakfast
16,French Harbor,HN,79.00,16.3500,-86.4333,"Fantasy Island Beach Resort, Dive & Marina"
17,Victoria,HK,70.25,22.2855,114.1577,Mini Hotel Central


In [19]:
# Add a heatmap of temperature for the vacation spots.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False,
             max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [20]:
# Add a heatmap of temperature for the vacation spots and marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,
             dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [21]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) 
              for index, row in hotel_df.iterrows()]
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))